In [5]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv('traffic.csv')  # Replace with your actual dataset path

# Preprocessing
X = df.drop('Accident_severity', axis=1)
y = df['Accident_severity']

# Convert categorical features to dummy/one-hot
X = pd.get_dummies(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

import os
# Create model directory if it doesn't exist
os.makedirs('model', exist_ok=True)

# Then save the model
with open('model/accident_model.pkl', 'wb') as f:
    pickle.dump((model, X.columns), f)



Model Accuracy: 0.84


In [7]:
app_code = '''
from flask import Flask, request, render_template
import pickle
import pandas as pd

app = Flask(__name__)

# Load the trained model and columns
with open('../model/accident_model.pkl', 'rb') as f:
    model, model_columns = pickle.load(f)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.form.to_dict()
        df = pd.DataFrame([data])
        df = pd.get_dummies(df)
        df = df.reindex(columns=model_columns, fill_value=0)

        prediction = model.predict(df)[0]
        return render_template('index.html', prediction=f'Predicted Severity: {prediction}')
    except Exception as e:
        return str(e)

if __name__ == '__main__':
    app.run(debug=True)
'''

# Save it as app.py
with open('app.py', 'w') as f:
    f.write(app_code)

print("✅ app.py created.")


✅ app.py created.
